# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [47]:
import numpy as np
import pandas as pd
import math
import xlsxwriter
import requests
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [34]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [35]:
from my_secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock

METHOD IS DEPRECIATED, USE YFINANCE INSTEAD

In [36]:
# symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
# print("status code: " + str(requests.get(api_url).status_code))
# data = requests.get(api_url).json()
# print(data)

In [37]:
# tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL']

# Fetching data
# data = yf.download(tickers, period='1d')

# Displaying the closing prices and market cap
# for ticker in tickers:
#     stock = yf.Ticker(ticker)
#     close_price = data['Close'][ticker].iloc[0]
#     market_cap = stock.info['marketCap']
#     print(f"{ticker} - Close Price: {close_price}, Market Cap: {market_cap}")


data = yf.download(tickers, period='1d')
stock = yf.Ticker('AAPL')
stock
dir(stock)

[*********************100%%**********************]  4 of 4 completed


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_analysis',
 '_data',
 '_download_options',
 '_earnings',
 '_earnings_dates',
 '_expirations',
 '_fast_info',
 '_fetch_ticker_tz',
 '_financials',
 '_fundamentals',
 '_get_ticker_tz',
 '_holders',
 '_isin',
 '_lazy_load_price_history',
 '_news',
 '_options2df',
 '_price_history',
 '_quote',
 '_shares',
 '_tz',
 '_underlying',
 'actions',
 'analyst_price_target',
 'balance_sheet',
 'balancesheet',
 'basic_info',
 'calendar',
 'capital_gains',
 'cash_flow',
 'cashflow',
 'dividends',
 'earnings',
 'earnings_dates',
 'earnings_forecasts',
 'earnings_trend',
 'fast_info',
 'financials',
 'get_actions',
 'get

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [38]:
price = stock.history(period='1d')['Close'].iloc[0]
print('Close Price: $', price)
market_cap = stock.info['marketCap']
print('Market Cap: ', market_cap/10**9, 'Billion')

Close Price: $ 226.33999633789062
Market Cap:  3470.720237568 Billion


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [39]:
my_columns = ['ticker', 'stock_price', 'market_cap', 'number_of_shares_to_buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,ticker,stock_price,market_cap,number_of_shares_to_buy


In [41]:
new_row = pd.Series(
    [
        'AAPL',
        price,
        market_cap,
        'N/A'
    ],
    index=my_columns
)

# Convert the Series to a DataFrame and concatenate it with the existing DataFrame
pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)

,ticker,stock_price,market_cap,number_of_shares_to_buy
0,AAPL,226.339996,3470720237568,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [46]:
final_dataframe = pd.DataFrame(columns=my_columns)
for ticker in stocks['Ticker']:
    try:
        stock = yf.Ticker(ticker)
        history = stock.history(period='1d')
        
        if not history.empty:
            price = history['Close'].iloc[0]
            market_cap = stock.info['marketCap']
        else:
            price = None
            market_cap = None

        new_row = pd.Series(
            [
                ticker,
                price,
                market_cap,
                'N/A'
            ],
            index=my_columns
        )
        
        # Append the new row to the DataFrame
        final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
    except:
        continue

ABC: No data found, symbol may be delisted
$ABMD: possibly delisted; No price data found  (period=1d)


$ABMD: possibly delisted; No price data found  (period=1d)


ALXN: No data found, symbol may be delisted
ANTM: No data found, symbol may be delisted
$ATVI: possibly delisted; No price data found  (period=1d)
BF.B: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
BLL: No data found, symbol may be delisted
BRK.B: No data found, symbol may be delisted
CERN: No data found, symbol may be delisted
COG: No data found, symbol may be delisted
CTL: No data found, symbol may be delisted
CTXS: No data found, symbol may be delisted
CXO: No data found, symbol may be delisted
DISCA: No data found, symbol may be delisted
DISCK: No data found, symbol may be delisted
DISH: No data found, symbol may be delisted
DRE: No data found, symbol may be delisted
ETFC: No data found, symbol may be delisted
FB: No data found, symbol may be delisted
FBHS: No data found, symbol may be delisted
FLIR: No data found, symbol may be delisted
FLT: No data found, symbol may be delisted
FRC: No data found, symbol may be delisted
HFC: 

$SIVB: possibly delisted; No price data found  (period=1d)


TIF: No data found, symbol may be delisted
TWTR: No data found, symbol may be delisted
VAR: No data found, symbol may be delisted
VIAC: No data found, symbol may be delisted
WLTW: No data found, symbol may be delisted
XLNX: No data found, symbol may be delisted


In [49]:
final_dataframe

,ticker,stock_price,market_cap,number_of_shares_to_buy
0,A,126.370003,36869836800,N/A
1,AAL,11.04,7243233280,N/A
2,AAP,58.720001,3501021440,N/A
3,AAPL,226.339996,3470720237568,N/A
4,ABBV,167.25,295341752320,N/A
...,...,...,...,...
499,YUM,129.119995,36364320768,N/A
500,ZBH,108.010002,22220681216,N/A
501,ZBRA,310.51001,15966238720,N/A
502,ZION,42.490002,6273861632,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [54]:
final_dataframe = pd.DataFrame(columns=my_columns)
def fetch_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        history = stock.history(period='1d')
        
        if not history.empty:
            price = history['Close'].iloc[0]
            market_cap = stock.info['marketCap']
        else:
            price = None
            market_cap = None
        
        return pd.Series(
            [
                ticker,
                price,
                market_cap,
                'N/A'
            ],
            index=my_columns
        )
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.Series([ticker, None, None, 'N/A'], index=my_columns)

# Using ThreadPoolExecutor to fetch data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(fetch_stock_data, ticker) for ticker in stocks['Ticker']]
    
    for future in as_completed(futures):
        final_dataframe = pd.concat([final_dataframe, future.result().to_frame().T], ignore_index=True)

$ABMD: possibly delisted; No price data found  (period=1d)


$ABMD: possibly delisted; No price data found  (period=1d)


ABC: No data found, symbol may be delisted
ALXN: No data found, symbol may be delisted
ANTM: No data found, symbol may be delisted
ATVI: No data found, symbol may be delisted
BF.B: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
BLL: No data found, symbol may be delisted
BRK.B: No data found, symbol may be delisted
CERN: No data found, symbol may be delisted
COG: No data found, symbol may be delisted
CTL: No data found, symbol may be delisted
CTXS: No data found, symbol may be delisted
CXO: No data found, symbol may be delisted
DISCK: No data found, symbol may be delisted
DISCA: No data found, symbol may be delisted
DISH: No data found, symbol may be delisted
DRE: No data found, symbol may be delisted
ETFC: No data found, symbol may be delisted
FBHS: No data found, symbol may be delisted
FB: No data found, symbol may be delisted


Error fetching data for FISV: 'marketCap'


FLT: No data found, symbol may be delisted
FLIR: No data found, symbol may be delisted
FRC: No data found, symbol may be delisted
HFC: No data found, symbol may be delisted
INFO: No data found, symbol may be delisted
KSU: No data found, symbol may be delisted
MXIM: No data found, symbol may be delisted
NBL: No data found, symbol may be delisted
MYL: No data found, symbol may be delisted
NLSN: No data found, symbol may be delisted
NLOK: No data found, symbol may be delisted
PBCT: No data found, symbol may be delisted
PEAK: No data found, symbol may be delisted
PKI: No data found, symbol may be delisted
PXD: No data found, symbol may be delisted
RE: No data found, symbol may be delisted
$SIVB: possibly delisted; No price data found  (period=1d)


$SIVB: possibly delisted; No price data found  (period=1d)


TIF: No data found, symbol may be delisted
TWTR: No data found, symbol may be delisted
VAR: No data found, symbol may be delisted
VIAC: No data found, symbol may be delisted
WLTW: No data found, symbol may be delisted
XLNX: No data found, symbol may be delisted


In [68]:
final_dataframe[(final_dataframe['stock_price'].notna()) & (final_dataframe['market_cap'].notna())]


,ticker,stock_price,market_cap,number_of_shares_to_buy
0,AAPL,226.339996,3470720237568,N/A
1,AAL,11.04,7243233280,N/A
2,AAP,58.720001,3501021440,N/A
3,A,126.370003,36869836800,N/A
5,ABBV,167.25,295341752320,N/A
...,...,...,...,...
500,YUM,129.119995,36364320768,N/A
501,ZBH,108.010002,22220681216,N/A
502,ZBRA,310.51001,15966238720,N/A
503,ZION,42.490002,6273861632,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: